In [1]:
import pandas as pd
import numpy as np
import scipy.stats as ss
import statsmodels.api as sm
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

In [2]:
portfolio = pd.read_excel(io="Workshop Data.xlsx",sheet_name="Portfolio")
HYGVol = pd.read_excel(io="Workshop Data.xlsx",sheet_name="Adj HYG")
LQDyas = pd.read_excel(io="Indexes and Spreads Data 01.09.xlsx",sheet_name="LQD")
HYGyas = pd.read_excel(io="Indexes and Spreads Data 01.09.xlsx",sheet_name="HYG")
TenYearYield = pd.read_excel(io="Indexes and Spreads Data 01.09.xlsx",sheet_name="10yUST Yields")
HYyield = pd.read_excel(io="Indexes and Spreads Data 01.09.xlsx",sheet_name="HY Index")
IGyield = pd.read_excel(io="Indexes and Spreads Data 01.09.xlsx",sheet_name="IG Index")

In [3]:
portfolio = portfolio.set_index("Date")
HYGVol = HYGVol.set_index("Date").sort_index()
HYGVol = HYGVol.loc[:,"Volume"]
LQDyas = LQDyas.set_index("Date").sort_index()
HYGyas = HYGyas.set_index("Date").sort_index()
TenYearYield = TenYearYield.set_index("Date").sort_index()
HYyield = HYyield.set_index("Date").sort_index()
IGyield = IGyield.set_index("Date").sort_index()

In [4]:
portfolio.loc[:,"HYGCumDiv"]=portfolio.loc[:,"HYG Dividends"][::-1].cumsum()[::-1]
portfolio.loc[:,"TotalValueHYG"]=portfolio.loc[:,"HYG Position"]+portfolio.loc[:,"HYGCumDiv"]
portfolio = portfolio.sort_index()

In [5]:
HYGCorrData = pd.DataFrame(portfolio.loc["2021-01-11":"2025-12-31","TotalValueHYG"]).merge(HYyield.loc["2021-01-11":"2025-12-31"],left_index=True,right_index=True)
HYGCorrData.loc[:,"TotalReturnsHYG"] = HYGCorrData["TotalValueHYG"].pct_change()
HYGCorrData = HYGCorrData.rename(columns={"PX_LAST":"HY_INDEX_PX","OAS_SOVEREIGN_CURVE":"HY_Index_OAS","INDEX_SPREAD_BENCHMARK":"HY_INDEX_Spread","INDEX_Z_SPREAD_BP":"HY_INDEX_Z_Spread"})
HYGCorrData.loc[:,"HY_Index_OAS_Diff"]=HYGCorrData["HY_Index_OAS"].diff()
HYGCorrData.loc[:,"HY_Index_Spread_Diff"]=(HYGCorrData["HY_INDEX_Spread"].diff())/100
HYGCorrData.loc[:,"HY_Index_Z_Spread_Diff"]=(HYGCorrData["HY_INDEX_Z_Spread"].diff())/100

HYGCorrData = HYGCorrData.loc["2021-01-11":"2025-12-31"].merge(TenYearYield.loc["2021-01-11":"2025-12-31","PX_LAST"],left_index=True,right_index=True)
HYGCorrData = HYGCorrData.rename(columns={"PX_LAST":"10y_UST_INDEX_PX"})
HYGCorrData.loc[:,"10yYieldDiff"] = HYGCorrData["10y_UST_INDEX_PX"].diff()

HYGCorrData = HYGCorrData.loc["2021-01-11":"2025-12-31"].merge(IGyield.loc["2021-01-11":"2025-12-31"],left_index=True,right_index=True)
HYGCorrData = HYGCorrData.rename(columns={"PX_LAST":"IG_INDEX_PX","OAS_SOVEREIGN_CURVE":"IG_Index_OAS","INDEX_SPREAD_BENCHMARK":"IG_INDEX_Spread","INDEX_Z_SPREAD_BP":"IG_INDEX_Z_Spread"})
HYGCorrData.loc[:,"IG_Index_OAS_Diff"]=HYGCorrData["IG_Index_OAS"].diff()
HYGCorrData.loc[:,"IG_Index_Spread_Diff"]=(HYGCorrData["IG_INDEX_Spread"].diff())/100
HYGCorrData.loc[:,"IG_Index_Z_Spread_Diff"]=(HYGCorrData["IG_INDEX_Z_Spread"].diff())/100

HYGCorrData = HYGCorrData.loc["2021-01-11":"2025-12-31"].merge(HYGVol.loc["2021-01-11":"2025-12-31"],left_index=True,right_index=True)
HYGCorrData = HYGCorrData.rename(columns={"Volume":"HYG_Volume"})

HYGCorrData = HYGCorrData.loc["2021-01-11":"2025-12-31"].merge(portfolio.loc["2021-01-11":"2025-12-31","SPY Position"]/-10,left_index=True,right_index=True)
HYGCorrData = HYGCorrData.rename(columns={"SPY Position":"SPY_LAST_PX"})
HYGCorrData.loc[:,"ReturnSPY"] = HYGCorrData["SPY_LAST_PX"].pct_change()

HYGCorrData = HYGCorrData.loc["2021-01-11":"2025-12-31"].merge(HYGyas.loc["2021-01-11":"2025-12-31","YAS_MOD_DUR"],left_index=True,right_index=True)
HYGCorrData = HYGCorrData.rename(columns={"YAS_MOD_DUR":"HYG_MOD_DUR_YAS"})

HYGCorrData=HYGCorrData.dropna()
HYGCorrData = HYGCorrData.sort_index(ascending=False)
HYGCorrData.tail()

,TotalValueHYG,HY_Index_OAS,HY_INDEX_PX,HY_INDEX_Spread,HY_INDEX_Z_Spread,TotalReturnsHYG,HY_Index_OAS_Diff,HY_Index_Spread_Diff,HY_Index_Z_Spread_Diff,10y_UST_INDEX_PX,...,IG_INDEX_Spread,IG_INDEX_PX,IG_INDEX_Z_Spread,IG_Index_OAS_Diff,IG_Index_Spread_Diff,IG_Index_Z_Spread_Diff,HYG_Volume,SPY_LAST_PX,ReturnSPY,HYG_MOD_DUR_YAS
Date,,,,,,,,,,,,,,,,,,,,,
2021-01-19,8744.551454,3.475033,2349.11,374.901344,375.5320,0.003064,-0.021835,-0.046587,-0.029160,1.0886,...,91.843802,3527.74,92.5690,-0.013045,-0.009529,-0.013193,24270496,383.764922,-0.001561,2.8835
2021-01-15,8717.841195,3.496868,2346.26,379.560074,378.4480,-0.001408,0.025426,0.019678,0.026033,1.0835,...,92.796750,3519.97,93.8883,0.008244,0.007388,0.008302,48134776,384.364921,0.003944,2.8757
2021-01-14,8730.130922,3.471442,2345.24,377.592235,375.8447,-0.000033,-0.069708,-0.073834,-0.075329,1.1292,...,92.057996,3514.54,93.0581,-0.005403,-0.007324,-0.005297,25484782,382.854922,-0.003540,2.8614
2021-01-13,8730.420672,3.541150,2340.98,384.975631,383.3776,0.003184,-0.017457,-0.020986,-0.025771,1.0832,...,92.790383,3522.23,93.5878,-0.015203,-0.013986,-0.015302,47710312,384.214922,0.000912,2.9157
2021-01-12,8702.710408,3.558607,2338.69,387.074278,385.9547,0.001463,0.033598,0.055994,0.049256,1.1291,...,94.189026,3500.83,95.1180,0.000282,-0.000835,0.000608,41746060,383.864922,0.013840,2.9117


In [6]:
HYGCorrData[["TotalReturnsHYG","HY_Index_OAS_Diff","HY_Index_Spread_Diff","HY_Index_Z_Spread_Diff","10yYieldDiff","HYG_Volume","ReturnSPY","HYG_MOD_DUR_YAS","IG_Index_OAS_Diff","IG_Index_Spread_Diff","IG_Index_Z_Spread_Diff"]].corr()

,TotalReturnsHYG,HY_Index_OAS_Diff,HY_Index_Spread_Diff,HY_Index_Z_Spread_Diff,10yYieldDiff,HYG_Volume,ReturnSPY,HYG_MOD_DUR_YAS,IG_Index_OAS_Diff,IG_Index_Spread_Diff,IG_Index_Z_Spread_Diff
TotalReturnsHYG,1.000000,-0.467247,-0.417997,-0.439307,-0.409321,-0.088236,0.015848,-0.011211,-0.462306,-0.469921,-0.454278
HY_Index_OAS_Diff,-0.467247,1.000000,0.956867,0.983724,-0.269423,0.173969,-0.002169,0.010979,0.751647,0.770311,0.751890
HY_Index_Spread_Diff,-0.417997,0.956867,1.000000,0.951640,-0.310320,0.175017,-0.005774,0.006552,0.726280,0.758324,0.727735
HY_Index_Z_Spread_Diff,-0.439307,0.983724,0.951640,1.000000,-0.319092,0.176045,-0.007081,0.008903,0.746788,0.761437,0.749716
10yYieldDiff,-0.409321,-0.269423,-0.310320,-0.319092,1.000000,0.008738,-0.019481,0.025894,-0.190722,-0.166129,-0.205695
HYG_Volume,-0.088236,0.173969,0.175017,0.176045,0.008738,1.000000,0.024158,0.188971,0.167366,0.175940,0.169628
ReturnSPY,0.015848,-0.002169,-0.005774,-0.007081,-0.019481,0.024158,1.000000,-0.027047,-0.010107,-0.008358,-0.009083
HYG_MOD_DUR_YAS,-0.011211,0.010979,0.006552,0.008903,0.025894,0.188971,-0.027047,1.000000,0.016360,0.014648,0.016521
IG_Index_OAS_Diff,-0.462306,0.751647,0.726280,0.746788,-0.190722,0.167366,-0.010107,0.016360,1.000000,0.965244,0.998390
IG_Index_Spread_Diff,-0.469921,0.770311,0.758324,0.761437,-0.166129,0.175940,-0.008358,0.014648,0.965244,1.000000,0.964086


**PCA ANALYSIS**

Let's run a standanrdize PCA, for the credit spreads block of information, variables

In [ ]:
X_Std = StandardScaler().fit_transform(HYGCorrData[["HY_Index_OAS_Diff","HY_Index_Spread_Diff","HY_Index_Z_Spread_Diff","IG_Index_OAS_Diff","IG_Index_Spread_Diff","IG_Index_Z_Spread_Diff"]])
pcav1 = PCA()
pcav1.fit(X_Std)

,"n_components n_components: int, float or 'mle', default=NoneNumber of components to keep.if n_components is not set all components are kept:: n_components == min(n_samples, n_features)If ``n_components == 'mle'`` and ``svd_solver == 'full'``, Minka'sMLE is used to guess the dimension. Use of ``n_components == 'mle'``will interpret ``svd_solver == 'auto'`` as ``svd_solver == 'full'``.If ``0 < n_components < 1`` and ``svd_solver == 'full'``, select thenumber of components such that the amount of variance that needs to beexplained is greater than the percentage specified by n_components.If ``svd_solver == 'arpack'``, the number of components must bestrictly less than the minimum of n_features and n_samples.Hence, the None case results in:: n_components == min(n_samples, n_features) - 1",None
,"copy copy: bool, default=TrueIf False, data passed to fit are overwritten and runningfit(X).transform(X) will not yield the expected results,use fit_transform(X) instead.",True
,"whiten whiten: bool, default=FalseWhen True (False by default) the `components_` vectors are multipliedby the square root of n_samples and then divided by the singular valuesto ensure uncorrelated outputs with unit component-wise variances.Whitening will remove some information from the transformed signal(the relative variance scales of the components) but can sometimeimprove the predictive accuracy of the downstream estimators bymaking their data respect some hard-wired assumptions.",False
,"svd_solver svd_solver: {'auto', 'full', 'covariance_eigh', 'arpack', 'randomized'}, default='auto'""auto"" : The solver is selected by a default 'auto' policy is based on `X.shape` and `n_components`: if the input data has fewer than 1000 features and more than 10 times as many samples, then the ""covariance_eigh"" solver is used. Otherwise, if the input data is larger than 500x500 and the number of components to extract is lower than 80% of the smallest dimension of the data, then the more efficient ""randomized"" method is selected. Otherwise the exact ""full"" SVD is computed and optionally truncated afterwards.""full"" : Run exact full SVD calling the standard LAPACK solver via `scipy.linalg.svd` and select the components by postprocessing""covariance_eigh"" : Precompute the covariance matrix (on centered data), run a classical eigenvalue decomposition on the covariance matrix typically using LAPACK and select the components by postprocessing. This solver is very efficient for n_samples >> n_features and small n_features. It is, however, not tractable otherwise for large n_features (large memory footprint required to materialize the covariance matrix). Also note that compared to the ""full"" solver, this solver effectively doubles the condition number and is therefore less numerical stable (e.g. on input data with a large range of singular values).""arpack"" : Run SVD truncated to `n_components` calling ARPACK solver via `scipy.sparse.linalg.svds`. It requires strictly `0 < n_components < min(X.shape)`""randomized"" : Run randomized SVD by the method of Halko et al... versionadded:: 0.18.0.. versionchanged:: 1.5 Added the 'covariance_eigh' solver.",'auto'
,"tol tol: float, default=0.0Tolerance for singular values computed by svd_solver == 'arpack'.Must be of range [0.0, infinity)... versionadded:: 0.18.0",0.0
,"iterated_power iterated_power: int or 'auto', default='auto'Number of iterations for the power method computed bysvd_solver == 'randomized'.Must be of range [0, infinity)... versionadded:: 0.18.0",'auto'
,"n_oversamples n_oversamples: int, default=10This parameter is only relevant when `svd_solver=""randomized""`.It corresponds to the additional number of random vectors to sample therange of `X` so as to ensure proper conditioning. See:func:`~sklearn.utils.extmath.randomized_svd` for more details... versionadded:: 1.1",10
,"power_iteration_normalizer power_iteration_normalizer: {'auto', 'QR', 'LU', 'none'}, default='auto'Power iteration normalizer for randomized S

In [21]:
pcav1.explained_variance_ratio_

array([8.64705430e-01, 1.15608067e-01, 1.02927543e-02, 6.49743558e-03,
       2.64985011e-03, 2.46462829e-04])

 - PC1: 86.47%
 - PC2: 11.56%
 - PC3: 1.03%
 - PC4: 0.65%
 - PC5: 0.26%
 - PC6: 0.02%

In [22]:
pcav1.components_

array([[ 0.41020194,  0.40276491,  0.40853447,  0.40841211,  0.41084293,
         0.40868327],
       [-0.40035717, -0.41956898, -0.40635262,  0.42872435,  0.36579948,
         0.42537012],
       [-0.2929223 ,  0.67904498, -0.42708475, -0.18860768,  0.44052055,
        -0.20263723],
       [ 0.23411254, -0.44604462,  0.16320325, -0.32117568,  0.70476013,
        -0.34606294],
       [ 0.7257851 , -0.04063596, -0.67464848,  0.09541364, -0.0808858 ,
        -0.02806889],
       [-0.06381493,  0.00575677,  0.06156198,  0.70821222, -0.01054117,
        -0.70030634]])

In [23]:
pd.DataFrame(pcav1.components_, columns=["HY_Index_OAS_Diff","HY_Index_Spread_Diff","HY_Index_Z_Spread_Diff","IG_Index_OAS_Diff","IG_Index_Spread_Diff","IG_Index_Z_Spread_Diff"])

,HY_Index_OAS_Diff,HY_Index_Spread_Diff,HY_Index_Z_Spread_Diff,IG_Index_OAS_Diff,IG_Index_Spread_Diff,IG_Index_Z_Spread_Diff
0,0.410202,0.402765,0.408534,0.408412,0.410843,0.408683
1,-0.400357,-0.419569,-0.406353,0.428724,0.365799,0.425370
2,-0.292922,0.679045,-0.427085,-0.188608,0.440521,-0.202637
3,0.234113,-0.446045,0.163203,-0.321176,0.704760,-0.346063
4,0.725785,-0.040636,-0.674648,0.095414,-0.080886,-0.028069
5,-0.063815,0.005757,0.061562,0.708212,-0.010541,-0.700306


Based on the loadings it is possible to observe that the PC1 have similar magnitudes and sign accross HG and IV, which could indicate a systematic credit factor. Or the macroeconomic aspects of how the bonds risks is perceived againts the US treasuries. 

For PC2, we can validate our hypothesis tested on the first jupyter submitted that the HY-IG factors sustain a relationship which helps our regression to explain HY ETF tracker returns. Such that when HY spread widens relative to IG PC2 increases, and than when IG widens relative to HY PC2 decreaes. This can be explained as the second row have different signs for HY columns againts IG columns. 

The other PC3-PC6 are less than 2% and can be attributed to noise.

**Hence we will keep PC1 and PC2 factors in our regressions.

In [ ]:
X_pca = pcav1.transform(X_Std)

In [25]:
HYGCorrData.loc[:,"Credit_PC1"]=X_pca[:,0]
HYGCorrData.loc[:,"Credit_PC2"]=X_pca[:,1]

In [28]:
HYGCorrData

,TotalValueHYG,HY_Index_OAS,HY_INDEX_PX,HY_INDEX_Spread,HY_INDEX_Z_Spread,TotalReturnsHYG,HY_Index_OAS_Diff,HY_Index_Spread_Diff,HY_Index_Z_Spread_Diff,10y_UST_INDEX_PX,...,IG_INDEX_Z_Spread,IG_Index_OAS_Diff,IG_Index_Spread_Diff,IG_Index_Z_Spread_Diff,HYG_Volume,SPY_LAST_PX,ReturnSPY,HYG_MOD_DUR_YAS,Credit_PC1,Credit_PC2
Date,,,,,,,,,,,,,,,,,,,,,
2025-12-31,10168.486998,2.663664,2914.49,291.500404,293.7437,-0.000608,-0.010172,-0.019988,-0.009100,4.1670,...,78.6033,0.005634,0.004432,0.005872,17457064,689.248370,-0.000435,2.8197,0.180173,0.533434
2025-12-30,10174.674513,2.673836,2914.22,293.499232,294.6537,0.000965,-0.037478,-0.043672,-0.043267,4.1219,...,78.0161,-0.003152,-0.009691,-0.003344,20146744,689.548359,-0.003224,2.8482,-0.923771,0.257137
2025-12-29,10164.862002,2.711314,2911.40,297.866454,298.9804,0.000474,0.001774,0.035216,0.001935,4.1102,...,78.3505,0.007974,0.008389,0.007976,22012766,691.778359,0.005947,2.8465,0.728513,0.336379
2025-12-26,10160.049494,2.709540,2908.63,294.344862,298.7869,-0.000215,0.037774,0.027729,0.042589,4.1277,...,77.5529,-0.003501,-0.003903,-0.003661,10922908,687.688360,0.006660,2.8596,0.293371,-0.754813
2025-12-24,10162.237014,2.671766,2909.19,291.571921,294.5280,0.001657,-0.007383,-0.019628,-0.007108,4.1335,...,77.9190,-0.006932,-0.007354,-0.006746,10533489,683.138361,0.001833,2.8713,-0.598985,-0.291486
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-01-19,8744.551454,3.475033,2349.11,374.901344,375.5320,0.003064,-0.021835,-0.046587,-0.029160,1.0886,...,92.5690,-0.013045,-0.009529,-0.013193,24270496,383.764922,-0.001561,2.8835,-1.219939,-0.315622
2021-01-15,8717.841195,3.496868,2346.26,379.560074,378.4480,-0.001408,0.025426,0.019678,0.026033,1.0835,...,93.8883,0.008244,0.007388,0.008302,48134776,384.364921,0.003944,2.8757,0.877845,0.178748
2021-01-14,8730.130922,3.471442,2345.24,377.592235,375.8447,-0.000033,-0.069708,-0.073834,-0.075329,1.1292,...,93.0581,-0.005403,-0.007324,-0.005297,25484782,382.854922,-0.003540,2.8614,-1.412717,0.657493


In [33]:
X = sm.add_constant(HYGCorrData[["Credit_PC1","Credit_PC2","10yYieldDiff","ReturnSPY","HYG_MOD_DUR_YAS"]])
Y = HYGCorrData["TotalReturnsHYG"]*100

model = sm.OLS(Y,X).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:        TotalReturnsHYG   R-squared:                       0.550
Model:                            OLS   Adj. R-squared:                  0.548
Method:                 Least Squares   F-statistic:                     302.8
Date:                Sun, 11 Jan 2026   Prob (F-statistic):          8.30e-212
Time:                        22:45:42   Log-Likelihood:                -234.09
No. Observations:                1244   AIC:                             480.2
Df Residuals:                    1238   BIC:                             510.9
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
===================================================================================
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
const              -0.0162      0.065     -0.250      0.803      -0.143       0.111
Credit_PC1         -0.1224      0.004    -32.375      0.000      -0.130      -0.115
Credit_PC2          0.0368      0.010      3.636      0.000       0.017       0.057
10yYieldDiff       -4.1105      0.140    -29.353      0.000      -4.385      -3.836
ReturnSPY           0.0097      0.765      0.013      0.990      -1.492       1.511
HYG_MOD_DUR_YAS     0.0110      0.018      0.619      0.536      -0.024       0.046
==============================================================================
Omnibus:                      105.496   Durbin-Watson:                   2.363
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              604.030
Skew:                          -0.066   Prob(JB):                    6.86e-132
Kurtosis:                       6.411   Cond. No.                         347.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

<u> Some observations: </u>

It appears the Credit_PC1 -> MacroEconomic spreads & Credit_PC2-> HY and IG Spreads Relative Relationships, alongside the Treasury rates (10y) are the most important significant drivers of the returns observe in the HYG returns. Based on the regression of the end of the file, it appears HYG volume/liquidity is also highly dependent on the independent variables tested.

The regression above appears to indicate that Equities returns explanation on High Yield ETF fixed income tracker returns is minimal, and Mod Duration of the High yield index is somewhat significant, as it is the sensitivity of this day.

In [34]:
X = sm.add_constant(HYGCorrData[["Credit_PC1","Credit_PC2","10yYieldDiff","HYG_MOD_DUR_YAS"]])
Y = HYGCorrData["TotalReturnsHYG"]*100

model = sm.OLS(Y,X).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:        TotalReturnsHYG   R-squared:                       0.550
Model:                            OLS   Adj. R-squared:                  0.549
Method:                 Least Squares   F-statistic:                     378.8
Date:                Sun, 11 Jan 2026   Prob (F-statistic):          4.01e-213
Time:                        22:47:13   Log-Likelihood:                -234.09
No. Observations:                1244   AIC:                             478.2
Df Residuals:                    1239   BIC:                             503.8
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
===================================================================================
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
const              -0.0161      0.065     -0.250      0.803      -0.143       0.111
Credit_PC1         -0.1224      0.004    -32.390      0.000      -0.130      -0.115
Credit_PC2          0.0368      0.010      3.638      0.000       0.017       0.057
10yYieldDiff       -4.1105      0.140    -29.371      0.000      -4.385      -3.836
HYG_MOD_DUR_YAS     0.0110      0.018      0.619      0.536      -0.024       0.046
==============================================================================
Omnibus:                      105.496   Durbin-Watson:                   2.363
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              604.021
Skew:                          -0.066   Prob(JB):                    6.89e-132
Kurtosis:                       6.411   Cond. No.                         63.6
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [35]:
X = sm.add_constant(HYGCorrData[["Credit_PC1","Credit_PC2","10yYieldDiff"]])
Y = HYGCorrData["TotalReturnsHYG"]*100

model = sm.OLS(Y,X).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:        TotalReturnsHYG   R-squared:                       0.550
Model:                            OLS   Adj. R-squared:                  0.549
Method:                 Least Squares   F-statistic:                     505.2
Date:                Sun, 11 Jan 2026   Prob (F-statistic):          1.99e-214
Time:                        22:47:19   Log-Likelihood:                -234.28
No. Observations:                1244   AIC:                             476.6
Df Residuals:                    1240   BIC:                             497.1
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
================================================================================
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
const            0.0236      0.008      2.837      0.005       0.007       0.040
Credit_PC1      -0.1224      0.004    -32.392      0.000      -0.130      -0.115
Credit_PC2       0.0368      0.010      3.642      0.000       0.017       0.057
10yYieldDiff    -4.1080      0.140    -29.373      0.000      -4.382      -3.834
==============================================================================
Omnibus:                      104.585   Durbin-Watson:                   2.363
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              602.108
Skew:                          -0.022   Prob(JB):                    1.79e-131
Kurtosis:                       6.408   Cond. No.                         38.4
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [36]:
X = sm.add_constant(HYGCorrData[["Credit_PC1","Credit_PC2","10yYieldDiff","ReturnSPY","HYG_Volume","HYG_MOD_DUR_YAS"]])
Y = HYGCorrData["TotalReturnsHYG"]*100

model = sm.OLS(Y,X).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:        TotalReturnsHYG   R-squared:                       0.551
Model:                            OLS   Adj. R-squared:                  0.549
Method:                 Least Squares   F-statistic:                     253.4
Date:                Sun, 11 Jan 2026   Prob (F-statistic):          2.65e-211
Time:                        22:47:23   Log-Likelihood:                -232.33
No. Observations:                1244   AIC:                             478.7
Df Residuals:                    1237   BIC:                             514.5
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
===================================================================================
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
const              -0.0282      0.065     -0.434      0.664      -0.156       0.099
Credit_PC1         -0.1238      0.004    -32.146      0.000      -0.131      -0.116
Credit_PC2          0.0372      0.010      3.676      0.000       0.017       0.057
10yYieldDiff       -4.1260      0.140    -29.442      0.000      -4.401      -3.851
ReturnSPY          -0.0375      0.765     -0.049      0.961      -1.538       1.463
HYG_Volume       9.731e-10    5.2e-10      1.872      0.061   -4.66e-11    1.99e-09
HYG_MOD_DUR_YAS     0.0046      0.018      0.254      0.800      -0.031       0.040
==============================================================================
Omnibus:                      107.084   Durbin-Watson:                   2.372
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              579.721
Skew:                          -0.152   Prob(JB):                    1.30e-126
Kurtosis:                       6.331   Cond. No.                     3.67e+09
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 3.67e+09. This might indicate that there are
strong multicollinearity or other numerical problems.
"""